# create regen pedal map

**Author:** binjian Xin<br>
**Date Created:** 2021/07/29<br>
**Last Modified:** 2021/07/29<br>
**Description:** create regen pedal map and visualize<br>

## Create the non-eco pedal map

read default pedal map (claiming to be eco) from data file

In [ ]:
import numpy as np
import pandas as pd

# pm_folder = "xinddpg-pm-all/"
# pm_folder = "ddpg-pm/"
pm_folder = "ddpg-ao-full/tables/"
pm_path = "../../data/" + pm_folder
pd_data0 = pd.read_csv(pm_path + "init_table_coastdown.csv", header=0, index_col=0)
# pd_data0 = pd.read_csv("../../data/xinddpg-pm-all/init_table_coastdown.csv", header=0, index_col=0)

## Visualization
### plotly
show default pedal map

### Preliminaries for plotly and chart studio

In [ ]:
import plotly.graph_objects as go

# sh_0, sh_1 = pd_data0.shape
# x, y =  np.linspace(0,1.0,sh_1), np.linspace(0,100/3.6, sh_0)
# y[1] = 7/3.6
# x = np.array([0, 2, 4, 8, 12, 16, 20, 24, 28, 32, 38, 44, 50, 62, 74, 86, 100])/100
x = pd_data0.columns.to_numpy()
y = pd_data0.index.to_numpy()
z = pd_data0.values
# # Download data set from plotly repo
# pts = np.loadtxt(np.DataSource().open('https://raw.githubusercontent.com/plotly/datasets/master/mesh_dataset.txt'))
# x, y, z = pts.T
# fig1 = go.Figure(data=[go.Mesh3d(x=x, y=y, z=z, color='lightpink', opacity=0.50)])
# fig1.show()

z0 = np.zeros(z.shape)
figure0 = go.Figure(
    data=[
        go.Surface(
            contours={
                "y": {"show": True, "start": 0, "end": 25, "size": 5, "color": "cyan"},
                "z": {
                    "show": True,
                    "start": -3000,
                    "end": 4600,
                    "size": 100,
                    "color": "blue",
                },
            },
            x=x,
            y=y,
            z=pd_data0.values,
        ),
        go.Surface(x=x, y=y, z=z0, opacity=0.4),
    ]
)
# figure.update_traces(contours_z=dict(show=True, usecolormap=True,
#                                   highlightcolor="limegreen", project_z=True))
# figure0.add_trace(go.Surface(x=x,y=y,z=z0, surfacecolor=np.ones(z0.shape)))
camera = dict(
    up=dict(x=0, y=0, z=1),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=-1.25, y=0.25, z=1.0)
    # eye=dict(x=-1.25,y=1.25,z=1.25)
)
figure0.update_layout(
    title="Default Pedal Map",
    autosize=False,
    scene_camera=camera,
    scene=dict(xaxis_title="pedal", yaxis_title="velocity", zaxis_title="torque"),
    width=700,
    height=700,
    margin=dict(l=65, r=50, b=65, t=90),
)

figure0.show()
figure0.write_image(pm_path + "table_init.png", engine="kaleido")

Now show individual pedal map in the optimcal episode


In [ ]:
import glob

datafolder = pm_path + "instant_table_ddpg-*.csv"


files = sorted(glob.glob(datafolder))

In [ ]:
import os

for file in files:
    pd_data1 = pd.read_csv(file, header=0, index_col=0)
    x = pd_data1.columns.to_numpy()
    y = pd_data1.index.to_numpy()
    z = pd_data1.values

    figure1 = go.Figure(
        data=[
            go.Surface(
                contours={
                    "y": {
                        "show": True,
                        "start": 0,
                        "end": 25,
                        "size": 5,
                        "color": "cyan",
                    },
                    "z": {
                        "show": True,
                        "start": -3000,
                        "end": 4600,
                        "size": 100,
                        "color": "blue",
                    },
                },
                x=x,
                y=y,
                z=pd_data1.values,
            ),
            go.Surface(x=x, y=y, z=z0, opacity=0.4),
        ]
    )
    # figure.update_traces(contours_z=dict(show=True, usecolormap=True,
    #                                   highlightcolor="limegreen", project_z=True))
    # figure1.update_layout(title=file[72:], autosize=False,
    figure1.update_layout(
        title=file[80:],
        autosize=False,
        scene=dict(xaxis_title="pedal", yaxis_title="velocity", zaxis_title="torque"),
        width=720,
        height=720,
        margin=dict(l=65, r=50, b=65, t=90),
    )
    camera = dict(
        up=dict(x=0, y=0, z=1),
        center=dict(x=0, y=0, z=0),
        eye=dict(x=-1.25, y=0.25, z=1.0)
        # eye=dict(x=-1.25,y=0.75,z=1.25)
    )
    figure1.update_layout(scene_camera=camera)

    # figure0.add_trace(go.Surface(x=x,y=y,z=z0, surfacecolor=np.ones(z0.shape)))
    # figure1.show()
    pm_image = os.path.splitext(file)[0] + ".png"
    figure1.write_image(pm_image, engine="kaleido")

In [ ]:
import imageio

imagedatafolder = pm_path + "instant_table_ddpg-*.png"
gif_path = pm_path + "ddpg_pm_variation.gif"
files = sorted(glob.glob(imagedatafolder))
end_pause = 2  # seconds to stay on last frame, create end_pause
frame_length = 0.5  # seconds between frames
start_of_episode_string = "-0.png"
with imageio.get_writer(gif_path, mode="I", duration=frame_length, loop=0) as writer:
    for idx, file in enumerate(files):
        if idx != 0 and start_of_episode_string in file:
            for i in range(0, int(end_pause / frame_length)):
                writer.append_data(imageio.imread(file_last))
        writer.append_data(imageio.imread(file))
        file_last = file

In [ ]:
from pygifsicle import optimize

gif_path = pm_path + "ddpg_pm_variation.gif"
gif_path_opt = pm_path + "ddpg_pm_variation_opt.gif"
optimize(gif_path, gif_path_opt)

In [ ]:
import imageio

imagedatafolder = pm_path + "instant_table_ddpg-*.png"
mp4_path = pm_path + "ddpg_pm_variation.mp4"
fps = 5
files = sorted(glob.glob(imagedatafolder))
end_pause = 1  # seconds to stay on last frame, create end_pause
frame_length = 0.2  # seconds between frames
start_of_episode_string = "-0.png"
with imageio.get_writer(mp4_path, fps=fps) as writer:
    for idx, file in enumerate(files):
        if idx != 0 and start_of_episode_string in file:
            for i in range(0, int(end_pause / frame_length)):
                writer.append_data(imageio.imread(file_last))
        writer.append_data(imageio.imread(file))
        file_last = file